# Forecasting Univariate Time Series

This tutorial introduces the basics of time series forecasting in sktime using univariate data.

**Duration:** ~15 minutes

## Learning objectives

By the end of this tutorial, you will be able to:
- Load and understand simple time series datasets
- Understand pandas structure in sktime
- Use the sktime forecasting API with `y` and `fh` arguments
- Work with relative and absolute forecast horizons
- Perform forecasting with exponential smoothing

## 1. Loading a Simple Dataset

Let's start by loading the classic airline passengers dataset, which contains monthly totals of international airline passengers from 1949 to 1960.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sktime.datasets import load_airline
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.utils.plotting import plot_series

# Load the airline dataset
y = load_airline()
print(f"Dataset shape: {y.shape}")
print(f"Dataset type: {type(y)}")
print(f"Index type: {type(y.index)}")
print(y.head())

## 2. Understanding pandas Structure in sktime

In sktime, univariate time series are represented as pandas Series with a time-based index.

In [ ]:
# Examine the structure
print("First 10 values:")
print(y.head(10))
print("\nIndex information:")
print(f"Index name: {y.index.name}")
print(f"Index frequency: {y.index.freq}")
print(f"Start date: {y.index[0]}")
print(f"End date: {y.index[-1]}")

# Plot the time series
plot_series(y, title="Airline Passengers Dataset")
plt.show()

## 3. sktime Forecasting API: y and fh Arguments

The core of sktime's forecasting API involves two key arguments:
- `y`: The time series data to forecast from
- `fh`: The forecast horizon (when to forecast)

Let's split our data into training and test sets:

In [ ]:
# Split data: use first 108 observations for training, last 36 for testing
y_train = y.iloc[:-36]
y_test = y.iloc[-36:]

print(
    f"Training data: {y_train.index[0]} to {y_train.index[-1]} ({len(y_train)} observations)"
)
print(
    f"Test data: {y_test.index[0]} to {y_test.index[-1]} ({len(y_test)} observations)"
)

# Plot training and test data
plot_series(y_train, y_test, labels=["Training", "Test"], title="Train/Test Split")
plt.show()

## 4. Forecast Horizons: Relative and Absolute

The forecast horizon (`fh`) can be specified in two ways:

### 4.1 Relative Forecast Horizon

Relative horizons specify how many steps ahead to forecast from the end of the training data.

In [ ]:
from sktime.forecasting.base import ForecastingHorizon

# Relative forecast horizon: 1 to 36 steps ahead
fh_relative = ForecastingHorizon(range(1, 37), is_relative=True)
print(f"Relative FH: {fh_relative}")

# Or more simply:
fh_simple = range(1, 37)
print(f"Simple FH: {list(fh_simple)}")

### 4.2 Absolute Forecast Horizon

Absolute horizons specify exact time points to forecast.

In [ ]:
# Absolute forecast horizon: specific dates
fh_absolute = ForecastingHorizon(y_test.index, is_relative=False)
print(f"Absolute FH: {fh_absolute.to_pandas()[:5]}...")  # Show first 5

# Or directly using the test index
fh_test_index = y_test.index
print(f"Test index as FH: {fh_test_index[:5]}...")  # Show first 5

## 5. Forecasting with Exponential Smoothing

Now let's create our first forecast using exponential smoothing, a simple but effective forecasting method.

In [ ]:
# Initialize the forecaster
forecaster = ExponentialSmoothing(
    trend="add",  # Additive trend
    seasonal="multiplicative",  # Multiplicative seasonality
    sp=12,  # Seasonal period of 12 months
)

print(f"Forecaster: {forecaster}")

### 5.1 Fit the Model

In [ ]:
# Fit the forecaster to training data
forecaster.fit(y_train)
print("Model fitted successfully!")

### 5.2 Make Predictions

In [ ]:
# Generate forecasts using relative horizon
y_pred = forecaster.predict(fh=range(1, 37))

print(f"Forecast shape: {y_pred.shape}")
print(f"Forecast index: {y_pred.index[:5]}...")  # Show first 5
print("\nFirst 5 predictions:")
print(y_pred.head())

### 5.3 Visualize Results

In [ ]:
# Plot the results
plot_series(
    y_train,
    y_test,
    y_pred,
    labels=["Training", "Actual", "Forecast"],
    title="Exponential Smoothing Forecast",
)
plt.legend()
plt.show()

### 5.4 Evaluate the Forecast

In [ ]:
from sktime.performance_metrics.forecasting import (
    mean_absolute_percentage_error,
    mean_squared_error,
)

# Calculate error metrics
mape = mean_absolute_percentage_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Mean Absolute Percentage Error (MAPE): {mape:.2%}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")

## 6. Alternative Forecast Approaches

Let's explore different ways to specify forecast horizons:

In [ ]:
# Method 1: Using specific steps
fh_specific = [1, 6, 12, 24, 36]  # 1, 6, 12, 24, 36 months ahead
y_pred_specific = forecaster.predict(fh=fh_specific)
print(f"Specific steps forecast: {y_pred_specific}")

# Method 2: Using absolute dates
future_dates = pd.date_range(
    start=y_train.index[-1] + pd.DateOffset(months=1), periods=12, freq="MS"
)
y_pred_absolute = forecaster.predict(fh=future_dates)
print(f"\nAbsolute dates forecast (first 5): {y_pred_absolute.head()}")

## Summary

In this tutorial, you learned:

1. **Data Loading**: How to load simple time series datasets in sktime
2. **Data Structure**: Understanding pandas Series structure for time series
3. **API Basics**: The fundamental `y` and `fh` arguments in sktime forecasting
4. **Forecast Horizons**: Difference between relative and absolute forecast horizons
5. **Model Fitting**: How to fit and predict with ExponentialSmoothing
6. **Evaluation**: Basic forecast evaluation metrics

## Next Steps

- Try the "Forecasting with Exogenous Variables" tutorial to learn about more complex datasets
- Explore the "Transformations" tutorial to learn about data preprocessing
- Check out "Cross-validation and Metrics" for more robust evaluation techniques